<a href="https://colab.research.google.com/github/satani99/fastai_22_2/blob/main/backprop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!wget https://github.com/mnielsen/neural-networks-and-deep-learning/raw/master/data/mnist.pkl.gz

--2023-07-18 04:20:23--  https://github.com/mnielsen/neural-networks-and-deep-learning/raw/master/data/mnist.pkl.gz
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mnielsen/neural-networks-and-deep-learning/master/data/mnist.pkl.gz [following]
--2023-07-18 04:20:23--  https://raw.githubusercontent.com/mnielsen/neural-networks-and-deep-learning/master/data/mnist.pkl.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17051982 (16M) [application/octet-stream]
Saving to: ‘mnist.pkl.gz’

mnist.pkl.gz        100%[===================>]  16.26M  --.-KB/s    in 0.1s    

2023-07-18 04:20:23 (133 MB/s) - 

In [4]:
import pickle, gzip, math, os, time, shutil, torch, matplotlib as mpl, numpy as np
from pathlib import Path
from torch import tensor
from fastcore.test import test_close
torch.manual_seed(42)

mpl.rcParams['image.cmap'] = 'gray'
torch.set_printoptions(precision=2, linewidth=125, sci_mode=False)
np.set_printoptions(precision=2, linewidth=125)

path_data = Path('/content')
path_gz = path_data/'mnist.pkl.gz'
with gzip.open(path_gz, 'rb') as f: ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin')
x_train, y_train, x_valid, y_valid = map(tensor, [x_train, y_train, x_valid, y_valid])

In [5]:
n, m = x_train.shape
c = y_train.max() + 1
n, m, c

(50000, 784, tensor(10))

In [7]:
nh = 50

In [8]:
w1 = torch.randn(m, nh)
b1 = torch.zeros(nh)
w2 = torch.randn(nh, 1)
b2 = torch.zeros(1)

In [9]:
def lin(x, w, b): return x@w + b


In [10]:
t = lin(x_valid, w1, b1)
t.shape

torch.Size([10000, 50])

In [11]:
def relu(x): return x.clamp_min(0.)

In [12]:
t = relu(t)
t

tensor([[ 0.00, 11.87,  0.00,  ...,  5.48,  2.14, 15.30],
        [ 5.38, 10.21,  0.00,  ...,  0.88,  0.08, 20.23],
        [ 3.31,  0.12,  3.10,  ..., 16.89,  0.00, 24.74],
        ...,
        [ 4.01, 10.35,  0.00,  ...,  0.23,  0.00, 18.28],
        [10.62,  0.00, 10.72,  ...,  0.00,  0.00, 18.23],
        [ 2.84,  0.00,  1.43,  ...,  0.00,  5.75,  2.12]])

In [13]:
def model(xb):
  l1 = lin(xb, w1, b1)
  l2 = relu(l1)
  return lin(l2, w2, b2)

In [14]:
res = model(x_valid)
res.shape

torch.Size([10000, 1])

In [15]:
res.shape, y_valid.shape

(torch.Size([10000, 1]), torch.Size([10000]))

In [16]:
(res-y_valid).shape

torch.Size([10000, 10000])

In [20]:
res[:,0].shape

torch.Size([10000])

In [21]:
res.squeeze().shape

torch.Size([10000])

In [22]:
(res[:,0]-y_valid).shape

torch.Size([10000])

In [24]:
y_train, y_valid = y_train.float(), y_valid.float()

preds = model(x_train)
preds.shape

torch.Size([50000, 1])

In [25]:
def mse(output, targ): return (output.squeeze() - targ).pow(2).mean()

In [26]:
mse(preds, y_train)

tensor(4308.76)

In [28]:
from sympy import symbols, diff
x, y = symbols('x y')
diff(x**2, x)

2*x

In [29]:
diff(3*x**2+9, x)

6*x

In [ ]:
def lin_grad(inp, out, w, b):
